In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import chardet
import os
import time
import ast

# ------------------ Helpers ------------------

def detect_encoding(filepath):
    with open(filepath, 'rb') as f:
        raw = f.read(10000)
    result = chardet.detect(raw)
    return result['encoding'] or 'utf-8'

def load_csv(filepath):
    try:
        encoding = detect_encoding(filepath)
        print(f"🔍 Detected encoding: {encoding}")
        return pd.read_csv(filepath, encoding=encoding).head(10).dropna(how='all')
    except UnicodeDecodeError:
        print("⚠️ Encoding issue detected. Retrying with UTF-8 and fallback open().")
        with open(filepath, mode='r', encoding='utf-8', errors='replace') as f:
            return pd.read_csv(f).head(10).dropna(how='all')

def detect_unpackable_columns(df, sample_size=5):
    unpackable = []
    for col in df.columns:
        samples = df[col].dropna().astype(str).head(sample_size)
        for val in samples:
            try:
                parsed = ast.literal_eval(val.strip())
                if isinstance(parsed, dict) and all(isinstance(v, dict) for v in parsed.values()):
                    unpackable.append(col)
                    break
            except:
                continue
    return unpackable

def unpack_column(df, col, prefix):
    rows = []
    for _, row in df.iterrows():
        raw = row[col]
        try:
            nested = ast.literal_eval(str(raw).strip())
        except:
            continue
        for subkey, subvals in nested.items():
            if not isinstance(subvals, dict):
                continue
            new_row = row.to_dict()
            new_row[f"{prefix}_key"] = subkey
            for k, v in subvals.items():
                new_row[f"{prefix}_{k}"] = v
            rows.append(new_row)
    return pd.DataFrame(rows)

# ------------------ Summary ------------------

def summarize_dataframe(df, group_label):
    summary = []
    for col in df.columns:
        s = df[col].dropna()
        col_stats = {
            "column": col,
            "count": s.count(),
            "unique": s.nunique(),
            "mean": round(s.mean(), 4) if pd.api.types.is_numeric_dtype(s) else "NA",
            "min": s.min() if pd.api.types.is_numeric_dtype(s) else "NA",
            "max": s.max() if pd.api.types.is_numeric_dtype(s) else "NA",
            "std_dev": round(s.std(), 4) if pd.api.types.is_numeric_dtype(s) else "NA",
            "most_freq": f"{s.mode().iloc[0]} ({s.value_counts().iloc[0]})" if not pd.api.types.is_numeric_dtype(s) and not s.empty else "NA",
            "group": group_label
        }
        summary.append(col_stats)
    return summary

# ------------------ Plots ------------------

def plot_columns(df, numeric_cols, categorical_cols, prefix):
    os.makedirs("Task_05_plots_without_grouping", exist_ok=True)
    
    selected_numeric_cols = ["Goals Home", "Away Goals", "home_possessions", "away_possessions"]
    
    for col in selected_numeric_cols:
        if col in df.columns:
            try:
                df[col].value_counts().head(10).plot(kind='bar', title=f"Top Values: {col}")
                plt.xlabel(col)
                plt.ylabel("Count")
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig(f"Task_05_plots_without_grouping/{prefix}_bar_{col.replace(' ', '_')}.png")
                plt.close()
            except Exception as e:
                print(f"⚠ Bar chart failed for {col}: {e}")
        else:
            print(f"⚠ Column '{col}' not found in dataset. Skipping plot.")
            
'''
    for col in selected_numeric_cols:
        if col in df.columns:
            try:
                df[col].dropna().astype(float).plot(kind='hist', bins=20, title=f"Histogram: {col}")
                plt.xlabel(col)
                plt.ylabel("Frequency")
                plt.tight_layout()
                plt.savefig(f"Task_05_plots_without_grouping/{prefix}hist{col}.png")
                plt.close()
            except Exception as e:
                print(f"⚠ Histogram failed for {col}: {e}")
                
'''

# ------------------ Main ------------------

if __name__ == "__main__":
    print("📊 PANDAS DESCRIPTIVE STATISTICS")
    filepath = input("Enter CSV path : ").strip()
    dataset_name = os.path.splitext(os.path.basename(filepath))[0]
    group_input = input("Enter group keys (comma-separated or leave blank): ").strip()
    group_keys = [g.strip() for g in group_input.split(",") if g.strip()]

    print("📥 Loading data...")
    df = load_csv(filepath)
    df = df[:10]

    print("🔍 Unpacking nested columns (if any)...")
    unpack_cols = detect_unpackable_columns(df)
    for col in unpack_cols:
        df = unpack_column(df, col, col.split("_")[0])
        df.drop(columns=[col], inplace=True)

    print("📊 Summarizing...")
    start = time.perf_counter()
    summaries = summarize_dataframe(df, group_label="full_dataset")

    if group_keys:
        grouped = df.groupby(group_keys)
        for group_val, group_df in grouped:
            label = ", ".join(f"{k}={v}" for k, v in zip(group_keys, group_val)) if isinstance(group_val, tuple) else f"{group_keys[0]}={group_val}"
            summaries.extend(summarize_dataframe(group_df, group_label=label))

    os.makedirs("Task_05_summaries", exist_ok=True)
    out_file = f"Task_05_summaries/summary_without_grouping_{dataset_name}.csv"
    pd.DataFrame(summaries).to_csv(out_file, index=False)
    print(f"✅ Summary saved to {out_file}")

    # Optional plotting
    if input("📈 Generate plots? (y/n): ").strip().lower() == 'y':
        num_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]
        cat_cols = [col for col in df.columns if col not in num_cols]
        plot_columns(df, num_cols, cat_cols, dataset_name)
        print("✅ Plots saved to Task_05_plots_without_grouping/")

    print(f"⏱ Done in {time.perf_counter() - start:.2f} seconds")

📊 PANDAS DESCRIPTIVE STATISTICS


Enter CSV path :  Premier_League.csv
Enter group keys (comma-separated or leave blank):  


📥 Loading data...
🔍 Detected encoding: ascii
🔍 Unpacking nested columns (if any)...
📊 Summarizing...
✅ Summary saved to Task_05_summaries/summary_without_grouping_Premier_League.csv


📈 Generate plots? (y/n):  y


✅ Plots saved to Task_05_plots_without_grouping/
⏱ Done in 2.90 seconds


In [2]:
# Make sure columns exist and are numeric
df["home_possessions"] = pd.to_numeric(df["home_possessions"], errors="coerce")
df["away_possessions"] = pd.to_numeric(df["away_possessions"], errors="coerce")

# Calculate absolute possession difference
df["possession_diff"] = (df["home_possessions"] - df["away_possessions"]).abs()

# Find the row with the maximum difference
max_diff_row = df.loc[df["possession_diff"].idxmax()]

print("📊 Match with the largest possession difference:")
print(max_diff_row)


📊 Match with the largest possession difference:
date                                                    28th May 2023
clock                                                          4:30pm
stadium                                            St. Mary's Stadium
attendance                                                     31,129
Home Team                                                 Southampton
Goals Home                                                          4
Away Team                                                   Liverpool
Away Goals                                                          4
home_possessions                                                 30.9
away_possessions                                                 69.1
home_shots                                                         15
away_shots                                                         30
home_on                                                            10
away_on                                   

In [3]:
# Ensure goal columns are numeric
df["Goals Home"] = pd.to_numeric(df["Goals Home"], errors="coerce")
df["Away Goals"] = pd.to_numeric(df["Away Goals"], errors="coerce")

# Total goals per match
df["total_goals"] = df["Goals Home"] + df["Away Goals"]

# Mean and standard deviation
mean_total_goals = df["total_goals"].mean()
std_total_goals = df["total_goals"].std()

print(f"🎯 Mean total goals per match: {mean_total_goals:.2f}")
print(f"📉 Standard deviation: {std_total_goals:.2f}")


🎯 Mean total goals per match: 3.30
📉 Standard deviation: 2.16


In [4]:
# Calculate mean possessions
mean_home_poss = df["home_possessions"].mean()
mean_away_poss = df["away_possessions"].mean()

print(f"🏠 Avg home possession: {mean_home_poss:.2f}%")
print(f"🚗 Avg away possession: {mean_away_poss:.2f}%")

if mean_home_poss > mean_away_poss:
    print("✅ Home teams have higher average possession.")
else:
    print("❌ Away teams have higher average possession.")


🏠 Avg home possession: 47.83%
🚗 Avg away possession: 52.17%
❌ Away teams have higher average possession.


In [6]:
# Drop rows with missing values in required columns
subset_df = df[["home_possessions", "Goals Home"]].dropna()

# Compute correlation
correlation = subset_df["home_possessions"].corr(subset_df["Goals Home"])

print(f"📈 Correlation between home possession and home goals: {correlation:.3f}")
if correlation > 0.3:
    print("✅ Moderate/Strong positive correlation.")
elif correlation > 0:
    print("🟡 Weak positive correlation.")
else:
    print("❌ No meaningful positive correlation.")


📈 Correlation between home possession and home goals: -0.262
❌ No meaningful positive correlation.


In [7]:
# Create high and low possession buckets
median_poss = df["home_possessions"].median()

high_poss_df = df[df["home_possessions"] >= median_poss]
low_poss_df = df[df["home_possessions"] < median_poss]

mean_goals_high = high_poss_df["Goals Home"].mean()
mean_goals_low = low_poss_df["Goals Home"].mean()

print(f"⚽ Avg home goals when possession ≥ {median_poss:.1f}%: {mean_goals_high:.2f}")
print(f"⚽ Avg home goals when possession < {median_poss:.1f}%: {mean_goals_low:.2f}")

if mean_goals_high > mean_goals_low:
    print("✅ Higher possession tends to lead to more goals at home.")
else:
    print("❌ Possession does not clearly correlate with scoring.")


⚽ Avg home goals when possession ≥ 49.6%: 2.00
⚽ Avg home goals when possession < 49.6%: 2.00
❌ Possession does not clearly correlate with scoring.
